# Emission RAG Architecture Project

## Install Necessary Modules

In [ ]:
pip install sentence-transformers
pip install duckdb

In [6]:
pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 46.4 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries

In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import duckdb

## Import Data

### Import downloaded carbon emission data in csv format, upload to postgres

In [28]:
df = pd.read_csv('owid-co2-data.csv')

#filter only the latest data
filtered_df = df[df['year'] > 2000]
filtered_df.head(5)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
251,Afghanistan,2001,AFG,20284303.0,1.102127e+10,0.007,0.000,1.069,0.022,2.098,...,NaN,0.079,0.0,0.0,0.001,0.0,15.772,4.670,NaN,NaN
252,Afghanistan,2002,AFG,21378123.0,1.880487e+10,0.011,0.001,1.341,0.272,25.432,...,NaN,0.079,0.0,0.0,0.001,0.0,20.735,5.132,NaN,NaN
253,Afghanistan,2003,AFG,22733054.0,2.107434e+10,0.010,0.000,1.560,0.219,16.302,...,NaN,0.078,0.0,0.0,0.001,0.0,21.423,5.521,NaN,NaN
254,Afghanistan,2004,AFG,23560656.0,2.233257e+10,0.010,0.000,1.237,-0.322,-20.669,...,NaN,0.078,0.0,0.0,0.001,0.0,20.057,5.354,NaN,NaN
255,Afghanistan,2005,AFG,24404574.0,2.539769e+10,0.006,0.000,1.890,0.652,52.719,...,NaN,0.078,0.0,0.0,0.001,0.0,20.766,6.204,NaN,NaN


In [10]:
import os
import config  # Import the config file

# Set environment variables using values from config.py
os.environ["POSTGRES_HOST"] = config.POSTGRES_HOST
os.environ["POSTGRES_DB"] = config.POSTGRES_DB
os.environ["POSTGRES_USER"] = config.POSTGRES_USER
os.environ["POSTGRES_PASSWORD"] = config.POSTGRES_PASSWORD
os.environ["POSTGRES_SCHEMA"] = config.POSTGRES_SCHEMA

In [11]:
print(os.getenv("POSTGRES_HOST"))  # Should print: your_postgres_host
print(os.getenv("POSTGRES_DB"))    # Should print: your_database_name
# print(os.getenv("POSTGRES_PASSWORD"))    # You Should never print: passwords or sensitive data

uclba-de25v2.cluster-cowglvndjvxv.eu-west-2.rds.amazonaws.com
postgres


In [12]:
# Environment variables for PostgreSQL connection
POSTGRES_HOST = os.getenv("POSTGRES_HOST", "localhost")
POSTGRES_DB = os.getenv("POSTGRES_DB", "postgres")
POSTGRES_USER = os.getenv("POSTGRES_USER", "postgres")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD", "postgres")
POSTGRES_PORT = os.getenv("POSTGRES_PORT", 5432) # leave at the default port
POSTGRES_SCHEMA = os.getenv("POSTGRES_SCHEMA", "public")

In [13]:
import os
import sqlalchemy
from sqlalchemy import create_engine

# Create PostgreSQL connection string
pg_conn_string = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the SQLAlchemy engine
pg_engine = create_engine(pg_conn_string, connect_args={"options": f"-c search_path={POSTGRES_SCHEMA}"})


In [32]:
schema_name = POSTGRES_SCHEMA
table_name = f"{schema_name}.example_table_4"

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
id SERIAL PRIMARY KEY,
example_column VARCHAR(255) NOT NULL);
"""

In [33]:
drop_table_query = f"""
DROP TABLE "carbon_data"

"""

# DROP TABLE "sales"
# DROP TABLE "customers"
# DROP TABLE "products"

In [34]:
import psycopg2
import pandas as pd
try:
    conn = psycopg2.connect(
        host=POSTGRES_HOST,
        dbname=POSTGRES_DB,
        user=POSTGRES_USER ,
        password=POSTGRES_PASSWORD,
        port=POSTGRES_PORT
    )    
    print("Connection successful!")
    
    # Set the schema if required
    with conn.cursor() as cur:
        cur.execute(f"SET search_path TO {POSTGRES_SCHEMA};")
        print(f"Schema set to {POSTGRES_SCHEMA}")

        # cur = conn.cursor()
        # cur.execute(create_table_query)
        cur.execute(drop_table_query)
        conn.commit()
        cur.close()

        print(f"Table {table_name} created successfully.")

except Exception as e:
    print("Error connecting to the database:", e)
finally:
    if 'conn' in locals() and conn:
        conn.close()
        print("Connection closed.")

Connection successful!
Schema set to schema_muhammadtribosnia24uclacuk
Error connecting to the database: table "carbon_data" does not exist

Connection closed.


In [35]:
pg_engine = create_engine("postgresql://muhammad.tribosnia.24@ucl.ac.uk:rzMwmH@uclba-de25v2.cluster-cowglvndjvxv.eu-west-2.rds.amazonaws.com:5432/postgres")

filtered_df.to_sql("carbon_data", pg_engine, schema ='schema_muhammadtribosnia24uclacuk', index = False)


83

### Acess newsdata.io to get latest news, use duckdb to store in in parquet file

In [1]:
import os
import config  # Import the config file
import requests

# Set environment variables using values from config.py
os.environ["API_KEY"] = config.newsio_api


# Environment variables for API connection
API_KEY = os.getenv("API_KEY")

'pub_769248ddb51ef8ab5c1766e49e025b83cc2c7'

In [ ]:
query = "carbon emissions"
url = f"https://newsdata.io/api/1/news?apikey={API_KEY}&q={query}&language=en"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    articles = data.get('results', [])[:5]  # Select only the first 5 articles
    for i, article in enumerate(articles, start=1):
        print(f"{i}. {article.get('title')}")
        print(f"   {article.get('link')}")
        print(f"   Source: {article.get('source_id')} | Published: {article.get('pubDate')}\n")
else:
    print("Error:", response.status_code, response.text)


In [ ]:
# Build the DataFrame
results = data['results']
df_news = pd.DataFrame([{
    'title': item.get('title'),
    'description': item.get('description'),
    'source': item.get('source_name'),
    'country': ', '.join(item.get('country', [])),
    'link': item.get('link'),
    'pubDate': item.get('pubDate')
} for item in results])

# Show the first 5 rows
df_news

In [5]:
import pandas as pd
df_news = pd.read_csv('news.csv')
df_news = df_news.drop(columns=["Unnamed: 0"], errors='ignore')
df_news

,title,description,source,country,link,pubDate
0,E-Drive for Automotive Market Exclusive Report...,InsightAce Analytic Pvt. Ltd. announces the re...,Openpr.com,germany,https://www.openpr.com/news/3944732/e-drive-fo...,2025-03-28 10:15:37
1,Kolkata's 'Sundarban Tramjatra' begins to prom...,Kolkata's 'Sundarban Tramjatra' begins to prom...,Hindustan Times,india,https://www.hindustantimes.com/cities/kolkata-...,2025-03-28 10:12:21
2,"India Cement Market Set for Robust Growth, Pro...",India Cement Market OverviewThe India Cement M...,Openpr.com,germany,https://www.openpr.com/news/3943967/india-ceme...,2025-03-28 10:11:53
3,"India's First Green Steel: ARS Steel, Epic Gro...","Chennai, 26-March-2025: In a pioneering step t...",Openpr.com,germany,https://www.openpr.com/news/3943919/india-s-fi...,2025-03-28 10:11:49
4,Morocco’s National Railway Office Teams Up wit...,In a landmark move set to transform Morocco’s ...,Travel And Tour World,united states of america,https://www.travelandtourworld.com/news/articl...,2025-03-28 10:07:06
5,Sustainable Floristry: Embracing Eco-Friendly ...,NaN,Deccan Herald,india,https://www.deccanherald.com/dhbrandspot-pr/su...,2025-03-28 10:03:42
6,Key Trends Shaping the Future Data Center Liqu...,How Are the key drivers contributing to the ex...,Openpr.com,germany,https://www.openpr.com/news/3944657/key-trends...,2025-03-28 10:02:46
7,Opinion: Carney's emissions cap flip-flop cast...,The Liberal party may have a new face but it s...,National Post,canada,https://nationalpost.com/opinion/opinion-carne...,2025-03-28 10:00:41
8,Opinion: Is Mark Carney 'Canada first' or net ...,A true believer in the existential necessity o...,Financial Post,canada,https://financialpost.com/opinion/opinion-is-m...,2025-03-28 10:00:13
9,Federal Judge Cancels Biden’s Last Gulf Lease ...,A federal district judge has issued a ruling a...,Oil Price,"united kingdom, united states of america, indi...",https://oilprice.com/Latest-Energy-News/World-...,2025-03-28 10:00:00


In [11]:
# 1. Save the DataFrame as a Parquet file
df_news.to_parquet("emission_news.parquet", index=False)
df_news

# 2. Connect to (or create) the DuckDB database
con = duckdb.connect("emission_news_data.duckdb")

# 3. Load the Parquet file into the DuckDB database as a table
con.execute("""
    CREATE TABLE emission_news AS
    SELECT * FROM read_parquet('emission_news.parquet')
""")




,title,description,source,country,link,pubDate
0,E-Drive for Automotive Market Exclusive Report...,InsightAce Analytic Pvt. Ltd. announces the re...,Openpr.com,germany,https://www.openpr.com/news/3944732/e-drive-fo...,2025-03-28 10:15:37
1,Kolkata's 'Sundarban Tramjatra' begins to prom...,Kolkata's 'Sundarban Tramjatra' begins to prom...,Hindustan Times,india,https://www.hindustantimes.com/cities/kolkata-...,2025-03-28 10:12:21
2,"India Cement Market Set for Robust Growth, Pro...",India Cement Market OverviewThe India Cement M...,Openpr.com,germany,https://www.openpr.com/news/3943967/india-ceme...,2025-03-28 10:11:53
3,"India's First Green Steel: ARS Steel, Epic Gro...","Chennai, 26-March-2025: In a pioneering step t...",Openpr.com,germany,https://www.openpr.com/news/3943919/india-s-fi...,2025-03-28 10:11:49
4,Morocco’s National Railway Office Teams Up wit...,In a landmark move set to transform Morocco’s ...,Travel And Tour World,united states of america,https://www.travelandtourworld.com/news/articl...,2025-03-28 10:07:06
5,Sustainable Floristry: Embracing Eco-Friendly ...,NaN,Deccan Herald,india,https://www.deccanherald.com/dhbrandspot-pr/su...,2025-03-28 10:03:42
6,Key Trends Shaping the Future Data Center Liqu...,How Are the key drivers contributing to the ex...,Openpr.com,germany,https://www.openpr.com/news/3944657/key-trends...,2025-03-28 10:02:46
7,Opinion: Carney's emissions cap flip-flop cast...,The Liberal party may have a new face but it s...,National Post,canada,https://nationalpost.com/opinion/opinion-carne...,2025-03-28 10:00:41
8,Opinion: Is Mark Carney 'Canada first' or net ...,A true believer in the existential necessity o...,Financial Post,canada,https://financialpost.com/opinion/opinion-is-m...,2025-03-28 10:00:13
9,Federal Judge Cancels Biden’s Last Gulf Lease ...,A federal district judge has issued a ruling a...,Oil Price,"united kingdom, united states of america, indi...",https://oilprice.com/Latest-Energy-News/World-...,2025-03-28 10:00:00


In [ ]:
# Optional: Verify the data
con.execute("SELECT * FROM emission_news LIMIT 5").fetchdf()

## Creating RAG Summary and Sentence Embeddings Pipeline

### Download carbon data files from postgres and emission-related news from duckdb

In [14]:
###carbon data
from sqlalchemy import text
pg_engine = create_engine("postgresql://muhammad.tribosnia.24@ucl.ac.uk:rzMwmH@uclba-de25v2.cluster-cowglvndjvxv.eu-west-2.rds.amazonaws.com:5432/postgres")

query = text("SELECT * FROM schema_muhammadtribosnia24uclacuk.carbon_data;")
# sales_data = pd.read_sql_query(query, conn)
df = pd.read_sql_query(query, pg_engine)
df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,2001,AFG,20284303.0,1.102127e+10,0.007,0.000,1.069,0.022,2.098,...,NaN,0.079,0.000,0.000,0.001,0.0,15.772,4.670,NaN,NaN
1,Afghanistan,2002,AFG,21378123.0,1.880487e+10,0.011,0.001,1.341,0.272,25.432,...,NaN,0.079,0.000,0.000,0.001,0.0,20.735,5.132,NaN,NaN
2,Afghanistan,2003,AFG,22733054.0,2.107434e+10,0.010,0.000,1.560,0.219,16.302,...,NaN,0.078,0.000,0.000,0.001,0.0,21.423,5.521,NaN,NaN
3,Afghanistan,2004,AFG,23560656.0,2.233257e+10,0.010,0.000,1.237,-0.322,-20.669,...,NaN,0.078,0.000,0.000,0.001,0.0,20.057,5.354,NaN,NaN
4,Afghanistan,2005,AFG,24404574.0,2.539769e+10,0.006,0.000,1.890,0.652,52.719,...,NaN,0.078,0.000,0.000,0.001,0.0,20.766,6.204,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860,Zimbabwe,2018,ZWE,15034457.0,2.678627e+10,0.558,0.037,11.205,1.379,14.030,...,NaN,0.107,0.001,0.001,0.002,0.0,35.722,18.689,-0.252,-2.252
5861,Zimbabwe,2019,ZWE,15271377.0,2.514642e+10,0.473,0.031,10.263,-0.942,-8.411,...,NaN,0.106,0.001,0.001,0.002,0.0,34.348,17.531,-0.027,-0.261
5862,Zimbabwe,2020,ZWE,15526888.0,2.317871e+10,0.496,0.032,8.495,-1.768,-17.231,...,NaN,0.105,0.001,0.001,0.002,0.0,31.323,15.775,0.612,7.209
5863,Zimbabwe,2021,ZWE,15797220.0,2.514009e+10,0.531,0.034,10.204,1.709,20.120,...,NaN,0.104,0.001,0.001,0.002,0.0,33.549,17.599,0.539,5.280


In [15]:
###emission-related news
df_news = con.execute("SELECT * FROM emission_news").fetchdf()
df_news

,rag_summary,embeddings
0,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.022816572338342667, 0.0530795119702816, -0..."
1,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.02335883118212223, 0.03286804258823395, -0..."
2,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.023150034248828888, 0.03248152881860733, -..."
3,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.018603665754199028, 0.04711500555276871, -..."
4,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.02122417651116848, 0.041178856045007706, -..."
...,...,...
5870,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.04372638091444969, 0.07870461791753769, 0.0..."
5871,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[-0.016487013548612595, -0.07614518702030182, ..."
5872,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.01761428453028202, 0.03581300377845764, 0.0..."
5873,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.012168467044830322, -0.0007799360901117325,..."


### Turn dataframe into RAG Summary

In [37]:
# Create rag_summary using all columns
df['rag_summary'] = df.apply(lambda row: (
    "[TYPE: CARBON DATA]\n"
    "Document: Full Carbon Data Entry\n" +
    "\n".join([f"{col}: {row[col]}" for col in df.columns if col != 'rag_summary' and col != 'embeddings']) +
    "\nSource: Structured Carbon Dataset"
), axis=1)

df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share,rag_summary
0,Afghanistan,2001,AFG,20284303.0,1.102127e+10,0.007,0.000,1.069,0.022,2.098,...,0.079,0.000,0.000,0.001,0.0,15.772,4.670,NaN,NaN,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
1,Afghanistan,2002,AFG,21378123.0,1.880487e+10,0.011,0.001,1.341,0.272,25.432,...,0.079,0.000,0.000,0.001,0.0,20.735,5.132,NaN,NaN,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
2,Afghanistan,2003,AFG,22733054.0,2.107434e+10,0.010,0.000,1.560,0.219,16.302,...,0.078,0.000,0.000,0.001,0.0,21.423,5.521,NaN,NaN,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
3,Afghanistan,2004,AFG,23560656.0,2.233257e+10,0.010,0.000,1.237,-0.322,-20.669,...,0.078,0.000,0.000,0.001,0.0,20.057,5.354,NaN,NaN,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
4,Afghanistan,2005,AFG,24404574.0,2.539769e+10,0.006,0.000,1.890,0.652,52.719,...,0.078,0.000,0.000,0.001,0.0,20.766,6.204,NaN,NaN,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860,Zimbabwe,2018,ZWE,15034457.0,2.678627e+10,0.558,0.037,11.205,1.379,14.030,...,0.107,0.001,0.001,0.002,0.0,35.722,18.689,-0.252,-2.252,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
5861,Zimbabwe,2019,ZWE,15271377.0,2.514642e+10,0.473,0.031,10.263,-0.942,-8.411,...,0.106,0.001,0.001,0.002,0.0,34.348,17.531,-0.027,-0.261,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
5862,Zimbabwe,2020,ZWE,15526888.0,2.317871e+10,0.496,0.032,8.495,-1.768,-17.231,...,0.105,0.001,0.001,0.002,0.0,31.323,15.775,0.612,7.209,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...
5863,Zimbabwe,2021,ZWE,15797220.0,2.514009e+10,0.531,0.034,10.204,1.709,20.120,...,0.104,0.001,0.001,0.002,0.0,33.549,17.599,0.539,5.280,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...


In [38]:
df['rag_summary'][0]

'[TYPE: CARBON DATA]\nDocument: Full Carbon Data Entry\ncountry: Afghanistan\nyear: 2001\niso_code: AFG\npopulation: 20284303.0\ngdp: 11021273088.0\ncement_co2: 0.007\ncement_co2_per_capita: 0.0\nco2: 1.069\nco2_growth_abs: 0.022\nco2_growth_prct: 2.098\nco2_including_luc: 1.84\nco2_including_luc_growth_abs: 0.736\nco2_including_luc_growth_prct: 66.622\nco2_including_luc_per_capita: 0.091\nco2_including_luc_per_gdp: 0.167\nco2_including_luc_per_unit_energy: 0.394\nco2_per_capita: 0.053\nco2_per_gdp: 0.097\nco2_per_unit_energy: 0.229\ncoal_co2: 0.07\ncoal_co2_per_capita: 0.003\nconsumption_co2: nan\nconsumption_co2_per_capita: nan\nconsumption_co2_per_gdp: nan\ncumulative_cement_co2: 1.975\ncumulative_co2: 73.519\ncumulative_co2_including_luc: 355.437\ncumulative_coal_co2: 11.921\ncumulative_flaring_co2: 5.956\ncumulative_gas_co2: 14.106\ncumulative_luc_co2: 795.186\ncumulative_oil_co2: 39.56\ncumulative_other_co2: nan\nenergy_per_capita: 236.892\nenergy_per_gdp: 0.423\nflaring_co2: 0.0

In [24]:
df_news['rag_summary'] = df_news.apply(lambda row: (
    "[TYPE: NEWS ARTICLE]\n"
    "Document: Climate & Emissions News\n"
    f"Title: {row['title']}\n"
    f"Description: {row['description']}\n"
    f"Source: {row['source']}\n"
    f"Country: {row['country']}\n"
    f"Published At: {row['pubDate']}\n"
    f"Link: {row['link']}\n"
    "Source: External News Feed"
), axis=1)

df_news



,title,description,source,country,link,pubDate,rag_summary
0,E-Drive for Automotive Market Exclusive Report...,InsightAce Analytic Pvt. Ltd. announces the re...,Openpr.com,germany,https://www.openpr.com/news/3944732/e-drive-fo...,2025-03-28 10:15:37,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
1,Kolkata's 'Sundarban Tramjatra' begins to prom...,Kolkata's 'Sundarban Tramjatra' begins to prom...,Hindustan Times,india,https://www.hindustantimes.com/cities/kolkata-...,2025-03-28 10:12:21,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
2,"India Cement Market Set for Robust Growth, Pro...",India Cement Market OverviewThe India Cement M...,Openpr.com,germany,https://www.openpr.com/news/3943967/india-ceme...,2025-03-28 10:11:53,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
3,"India's First Green Steel: ARS Steel, Epic Gro...","Chennai, 26-March-2025: In a pioneering step t...",Openpr.com,germany,https://www.openpr.com/news/3943919/india-s-fi...,2025-03-28 10:11:49,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
4,Morocco’s National Railway Office Teams Up wit...,In a landmark move set to transform Morocco’s ...,Travel And Tour World,united states of america,https://www.travelandtourworld.com/news/articl...,2025-03-28 10:07:06,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
5,Sustainable Floristry: Embracing Eco-Friendly ...,None,Deccan Herald,india,https://www.deccanherald.com/dhbrandspot-pr/su...,2025-03-28 10:03:42,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
6,Key Trends Shaping the Future Data Center Liqu...,How Are the key drivers contributing to the ex...,Openpr.com,germany,https://www.openpr.com/news/3944657/key-trends...,2025-03-28 10:02:46,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
7,Opinion: Carney's emissions cap flip-flop cast...,The Liberal party may have a new face but it s...,National Post,canada,https://nationalpost.com/opinion/opinion-carne...,2025-03-28 10:00:41,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
8,Opinion: Is Mark Carney 'Canada first' or net ...,A true believer in the existential necessity o...,Financial Post,canada,https://financialpost.com/opinion/opinion-is-m...,2025-03-28 10:00:13,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...
9,Federal Judge Cancels Biden’s Last Gulf Lease ...,A federal district judge has issued a ruling a...,Oil Price,"united kingdom, united states of america, indi...",https://oilprice.com/Latest-Energy-News/World-...,2025-03-28 10:00:00,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...


In [25]:
df_news['rag_summary'][0]

'[TYPE: NEWS ARTICLE]\nDocument: Climate & Emissions News\nTitle: E-Drive for Automotive Market Exclusive Report on the Latest Revenue and Future Scope\nDescription: InsightAce Analytic Pvt. Ltd. announces the release of a market assessment report on the "Global E-Drive for Automotive Market Size, Share & Trends Analysis Report Drive Type (Front Wheel Drive, Rear Wheel Drive, All Wheel Drive), Electric Vehicle Type (Battery\nSource: Openpr.com\nCountry: germany\nPublished At: 2025-03-28 10:15:37\nLink: https://www.openpr.com/news/3944732/e-drive-for-automotive-market-exclusive-report-on-the-latest\nSource: External News Feed'

### Turn RAG summary into sentence embeddings, and merge the RAG summary and the sentence embeddings from both dataset into a single dataset

In [39]:
import pandas as pd
import numpy as np
import re
import string
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Load SentenceTransformer Model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Generate Embeddings
carbondata_embeddings = embedder.encode(df["rag_summary"].tolist(), show_progress_bar=True)
news_embeddings = embedder.encode(df_news["rag_summary"].tolist(), show_progress_bar=True)

# Step 4: Store embeddings in a new column
df["embeddings"] = carbondata_embeddings.tolist()
df_news["embeddings"] = news_embeddings.tolist()

Batches:   0%|          | 0/184 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
### Merge RAG summary and embeddings
df_combined = pd.concat([
    df[['rag_summary', 'embeddings']],        # your carbon data
    df_news[['rag_summary', 'embeddings']]    # your news data
], ignore_index=True)

df_combined

,rag_summary,embeddings
0,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.022816572338342667, 0.0530795119702816, -0..."
1,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.02335883118212223, 0.03286804258823395, -0..."
2,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.023150034248828888, 0.03248152881860733, -..."
3,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.018603665754199028, 0.04711500555276871, -..."
4,[TYPE: CARBON DATA]\nDocument: Full Carbon Dat...,"[-0.02122417651116848, 0.041178856045007706, -..."
...,...,...
5870,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.04372638091444969, 0.07870461791753769, 0.0..."
5871,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[-0.016487013548612595, -0.07614518702030182, ..."
5872,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.01761428453028202, 0.03581300377845764, 0.0..."
5873,[TYPE: NEWS ARTICLE]\nDocument: Climate & Emis...,"[0.012168467044830322, -0.0007799360901117325,..."


In [2]:
import pandas as pd
import pandas as np

### Upload the Combined Data to Cloud Database (now use duckdb for testing)

In [5]:
import duckdb

# 1. Connect to (or create) the DuckDB database
con = duckdb.connect("combined_emission_data.duckdb")

# 2. Register the Pandas DataFrame as a temporary view
con.register("df_combined_view", df_combined)

# 3. Create a permanent table from that view
con.execute("""
    CREATE OR REPLACE TABLE combined_data AS
    SELECT * FROM df_combined_view
""")


NameError: name 'df_combined' is not defined

In [7]:
import duckdb

con = duckdb.connect("combined_emission_data.duckdb")

# List all tables in the database
con.execute("SHOW TABLES").fetchdf()


,name
0,emission_news


In [6]:
df_prepared_data = con.execute("SELECT * FROM df_combined_view").fetchdf()
df_prepared_data

CatalogException: Catalog Error: Table with name df_combined_view does not exist!
Did you mean "duckdb_views"?

LINE 1: SELECT * FROM df_combined_view
                      ^

## Build RAG Architecture using Gemini LLM

In [17]:
import os
import config  # Import the config file
import requests

# Set environment variables using values from config.py
os.environ["gemini_api_key"] = config.gemini_api


# Environment variables for API connection
gemini_api_key = os.getenv("gemini_api_key")

In [18]:
import google.generativeai as genai

# Replace with your API key
genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel("gemini-1.5-flash")


In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

document_embeddings = np.vstack(df_prepared_data["embeddings"].to_numpy())

def retrieve_context(query, top_k=3):
    query_embedding = embedder.encode([query])
    similarities = cosine_similarity(query_embedding, document_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return df_prepared_data['rag_summary'].iloc[top_indices].tolist()

def build_prompt(contexts, query):
    context_text = "\n".join(f"- {ctx}" for ctx in contexts)
    return f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context_text}

Question: {query}
Answer:"""

def ask_rag_with_gemini(query):
    contexts = retrieve_context(query)
    prompt = build_prompt(contexts, query)
    response = model.generate_content(prompt)
    return response.text.strip()


In [29]:
question = "What emission-related news was published about germany? give me the link as well and tell me when it is published"
answer = ask_rag_with_gemini(question)
print(answer)



There is no emission-related news specifically about Germany in the provided text.  The only German article is about the e-drive market for automobiles, which is related to emissions reduction but doesn't focus on emissions in Germany itself.  The article is from Openpr.com, published at 2025-03-28 10:15:37, and the link is https://www.openpr.com/news/3944732/e-drive-for-automotive-market-exclusive-report-on-the-latest


In [32]:

question = "how much is germany carbon emission"
answer = ask_rag_with_gemini(question)
print(answer)

Germany's carbon emissions varied across the years provided:

* **2008:** 856.607 (co2) and 844.444 (co2_including_luc) in metric tons.
* **2010:** 826.68 (co2) and 815.229 (co2_including_luc) in metric tons.
* **2015:** 800.844 (co2) and 794.87 (co2_including_luc) in metric tons.

The values differ slightly depending on whether land-use change is included in the calculation.
